In [19]:
#loading library
import numpy as np
import scipy as sp
import pandas as pd
from sklearn.cluster import KMeans

In [70]:
#loading data
data = pd.read_table('EMGaussian.data', header = None, sep='\s+', names = ('x1', 'x2'))
test = pd.read_table('EMGaussian.test', header = None, sep='\s+', names = ('x1', 'x2'))
X = np.array(data)

In [71]:
X

array([[  0.91029 ,   0.016192],
       [ -2.531   ,   6.6981  ],
       [  4.3602  ,   8.3541  ],
       [ -0.70489 ,  -0.51343 ],
       [ -5.7383  ,  -6.2088  ],
       [ -4.4819  ,  -4.7127  ],
       [  0.90842 ,   0.38202 ],
       [  4.4273  ,   5.8406  ],
       [ -5.5724  ,   3.9576  ],
       [  4.256   ,  -0.71465 ],
       [ -1.8763  ,   4.3208  ],
       [  3.9002  ,   4.9458  ],
       [ -1.6512  ,   3.1684  ],
       [  3.4097  ,  -2.8458  ],
       [  2.7509  ,  -3.7186  ],
       [  4.0017  ,  -5.6322  ],
       [ -1.3762  ,   2.2893  ],
       [  3.1672  ,  -2.0876  ],
       [  2.5706  ,   7.835   ],
       [  4.1783  ,   0.46918 ],
       [ -1.4094  ,   1.6812  ],
       [  3.3032  ,  -0.23407 ],
       [ -0.6899  ,   5.6732  ],
       [  4.4551  ,   0.91978 ],
       [  0.43072 ,   6.1793  ],
       [  4.2493  ,   7.8125  ],
       [ -2.2102  ,   5.1359  ],
       [  4.5083  ,   6.8181  ],
       [ -1.5381  ,   4.4656  ],
       [  3.8053  ,  -0.85611 ],
       [ -

In [164]:
#initialization
K = 4
[N,d] = data.shape #dimension of x
kmean_res = KMeans(init = 'k-means++', n_clusters=4).fit(data)
lmu = kmean_res.cluster_centers_
lsigma = [np.eye(2,2) for k in range(K)]
lpi = [1/K for k in range(K)]





In [85]:
k=2
i=2
lsigma[k]
lmu[k,:]
sp.stats.multivariate_normal

array([-3.78479953, -4.21639713])

In [132]:
k = 2
i = 1
lsigma[k]
sp.stats.multivariate_normal.pdf(X[i,:],lmu[k,:], lsigma[k])

4.49100440770126e-28

In [165]:
#E-step
#lpi is 1d-array; lmu, X are 2d-arrays; lsig is a list
Q = np.zeros((N,K))
condition = False
count = 0
tol=0.01

while(condition==False & count < 10):
    
    count += 1
    lmu_past=lmu
#    lsigma_past=lsigma

    for i in range(N):
        denom=0
        for k in range(K):
            denom=denom+lpi[k]*sp.stats.multivariate_normal.pdf(X[i,:],lmu[k,:], lsigma[k])
        for k in range(K):
            Q[i,k]=lpi[k]*sp.stats.multivariate_normal.pdf(X[i,:],lmu[k,:], lsigma[k])/denom

    #M-step
    for k in range(K):
        #compute mu
        lmu[k,:]= np.sum(Q[:,k].reshape(-1,1)*X)/np.sum(Q[:,k])
        #compute sigma
        diagQ = np.diag(Q[:,k])
        res = np.dot(np.transpose((X-lmu[k])), diagQ)
        res2 = np.dot(res, X)
        lsigma[k]=res2 /np.sum(Q)
        #compute pi
        lpi[k]=np.sum(Q[:,k])/np.sum(Q)
        
    
        #if (np.sum(np.abs(lmu-lmu_past))<tol):
            #condition=True
        
    

ValueError: the input matrix must be positive semidefinite

In [161]:
lsigma

[array([[ 3.18533044, -2.269394  ],
        [-3.46180831,  3.75035278]]), array([[ 2.72839693, -4.06993297],
        [-1.18142449,  2.87899831]]), array([[-4.09335226, -5.60104025],
        [-3.03566516, -2.88756747]]), array([[-3.13940628, -3.64533195],
        [-2.74213704, -3.10639562]])]

In [155]:
A = np.arange(9).reshape(3,3)
B = [1, 2, 3]
t = np.dot(A-B, np.eye(3,3))
np.dot(t,B)

array([ -6.,  12.,  30.])

In [162]:
count

1

In [117]:
#M-step
for k in range(K):
    #compute mu
    lmu[k,:]= np.sum(Q[:,k].reshape(-1,1)*X)/np.sum(Q[:,k])
    #compute sigma
    diff = X- lmu[k]
    diff=np.sqrt(Q[:,k].reshape(-1,1))*diff
    num=np.zeros((2,2))
    denom=np.sum(Q[:,k])
    for i in range(N):
        num=num+np.dot(np.transpose(diff[i,:]), diff[i,:])
    #num is a 2x2 array, denom a float
    lsigma[k]=num/denom
    #compute pi
    lpi[k]=np.sum(Q[:,k])/np.sum(Q)
    
    
        
    
        

In [108]:
k=1
np.dot(np.transpose((X - lmu[k])),(X - lmu[k]))

array([[  6811.90148996,   2270.04417975],
       [  2270.04417975,  10389.73236583]])

In [101]:
lmu[1]

array([ 0.70680149,  0.70680149])

In [17]:
mu0
sigma0

array([[ 0.70746067,  0.        ],
       [ 0.        ,  0.70746067]])